### Preliminaries (See Scrape.ipynb for more details)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait   
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver import ActionChains as AC   
from selenium.webdriver.common.keys import Keys  
import time
from tqdm.notebook import tqdm

opciones=Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False) 
opciones.add_argument('--start-maximized')
#opciones.add_argument('--incognito')


In [2]:
PATH=ChromeDriverManager().install()
driver=webdriver.Chrome(PATH, options=opciones) 

<ipython-input-2-0b7e44bbe702>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(PATH, options=opciones)


In [3]:
#find element
f = lambda x, y=driver : y.find_element(By.XPATH, x)
#find elements
fs = lambda x, y=driver : y.find_elements(By.XPATH, x)
#find and prepare to click
fc = lambda x, y=driver : y.find_element(By.XPATH, x).click()

### List of Best Selling Artists

##### Open Page

In [4]:
top_artists = 'https://en.wikipedia.org/wiki/List_of_best-selling_music_artists'
driver.get(top_artists)
time.sleep(2)

##### Table Headers

In [5]:
headers = [j.text for j in fs('//*[@id="mw-content-text"]/div[1]/table[1]/thead/tr/th')]
time.sleep(2)
headers[-2] = headers[-2].split('\n')[0]

##### Table Body

In [6]:
all_tables = [[[f('th',i).text] +[j.text for j in fs('td',i)] for i in fs('tbody/tr',k)] for k in fs('//*[@id="mw-content-text"]/div//table')[:5]] # 5 different tables
values = [rows for table in all_tables for rows in table] # flatten to one single table
artists = pd.DataFrame(values, columns = headers) # export to dataframe

##### Cleaning Data

In [7]:
# Remove footnotes/references
for i in ['Period active','Release-year of first charted record','Genre','Claimed sales']:
    artists[i] = artists[i].apply(lambda x : x.split('[')[0])

# Remove useless/uncertain info
artists['Total certified units'] = artists['Total certified units'].apply(lambda x : x.strip('show\n\n'))
artists['Country'] = artists['Country'].apply(lambda x : x.split('\n')[0])

# New Active column if presently active
artists['Active'] = artists['Period active'].apply(lambda x :'present' in x.lower())

# Make easier to use active period columns
def present(y):
    '''
    Returns year if band is not active, else NA
    '''
    try:
        int(y.split('–')[1])
        return y.split('–')[1]
    except:
        return pd.NA
artists['Start Year'] = artists['Period active'].apply(lambda x : x.split('–')[0])
artists['End Year'] = artists['Period active'].apply(present)
artists.drop(columns='Period active',inplace=True)

# Turn numerical values into machine-readable number data type
artists['Total certified units'] = artists['Total certified units'].apply( lambda x : float(x.split('m')[0]) * 1000000)
artists['Claimed sales'] = artists['Claimed sales'].apply( lambda x : float(x.split('m')[0]) * 1000000)

# clear spaces in column names
x = list(map(lambda x : x.replace(' ','_'),artists.columns))
artists = artists.rename(columns= {i: x[j] for j, i in enumerate(artists.columns)})

##### Exporting Genres

In [9]:
# Unique genres
genres = set()
for i in artists['Genre'].unique():
    x = i.split('/')
    for j in x:
        j = j.strip()
    for j in x:
        genres.add(j.lower())
Genres = pd.DataFrame(list(genres),columns=['Genre'])
Genres['genre_id'] = list(range(1,len(Genres)+1)) # assign id to each genre
Genres.to_parquet('../data/wikitop100list/genres.parquet',index=False) # export

#### Creating Many-to-Many Relation between Genres/Artists

In [10]:
# Intermediary table artist to genre
artist_genre = pd.DataFrame(artists['Artist'])

# Truth table for belonging-in-genre for each artist
for j,i in enumerate(genres):
    artist_genre[j+1] = artists['Genre'].apply(lambda x : i in x.lower())

# Assign list to each artist containing the unique genre ids
artist_genre['ids'] = ' '
for l,i in enumerate(artist_genre.itertuples()):
    genre_list = []
    for k,j in enumerate(i[2:-1]):
        if j:
            genre_list.append(k+1)
    artist_genre.at[ l , 'ids'] = genre_list

artist_genre = artist_genre[['Artist','ids']] # Drop truth table
artist_genre = artist_genre.explode('ids') # Explode the id list
artist_genre.to_parquet('../data/wikitop100list/artist_genre.parquet',index=False) # Export table

artists.drop(columns='Genre',inplace=True) # No longer needed
artists.to_parquet('../data/wikitop100list/artists.parquet',index=False) # Export table

In [11]:
x = pd.read_parquet('../data/wikitop100list/artists.parquet') # Test
x.head()

,Artist,Country,Release-year_of_first_charted_record,Total_certified_units,Claimed_sales,Active,Start_Year,End_Year
0,The Beatles,United Kingdom,1962,290600000.0,600000000.0,False,1960,1970
1,Elvis Presley,United States,1956,231800000.0,500000000.0,False,1953,1977
2,Michael Jackson,United States,1971,279900000.0,400000000.0,False,1964,2009
3,Elton John,United Kingdom,1970,208200000.0,300000000.0,True,1962,None
4,Queen,United Kingdom,1973,184200000.0,300000000.0,True,1971,None
